In [ ]:
import requests
import json
import pandas as pd
import altair as alt

In [ ]:
# Instituts-Kürzel
acronym = "HHI"

# Jahr von (min) bis (max). Für die Einschränkung auf ein Jahr, bitte den gleichen Wert für date_min und date_max einfügen
#date = "2021"
date_min = "2019" 
date_max = "2019"


url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?query=institute:("+acronym+")&configuration=researchoutputs&f.dateIssued=%5B"+date_min+"%20TO%20"+date_max+"%5D,equals"

r = requests.request("GET", url)
data = r.text
data = json.loads(data)
#print(data)
anzahl = data["_embedded"]["searchResult"]["page"]["totalElements"]
anzahl = int(anzahl)
print(anzahl)
pages = data["_embedded"]["searchResult"]["page"]["totalPages"]
pages = int(pages)
#print(pages)
object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

pub_uuid = []
pub_name = []
pub_href = []
pub_type = []
pub_journal = []
pub_journalAuth = []

object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

for j in range(len(object)):
    pub_uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
    pub_name.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["name"])
    pub_href.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_links"]["indexableObject"]["href"])
    #pub_journal.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["value"])
    #pub_journalAuth.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["authority"])
    pub_type.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.type"][0]["value"])

    
if pages > 1:
    for l in range(pages-1):
        url = data["_embedded"]["searchResult"]["_links"]["next"]["href"]
        r = requests.request("GET", url)
        #print(l)
        data = r.text
        data = json.loads(data)
        object = data["_embedded"]["searchResult"]["_embedded"]["objects"]
        for j in range(len(object)):
            pub_uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
            pub_name.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["name"])
            pub_href.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_links"]["indexableObject"]["href"])
            #pub_journal.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["value"])
            #pub_journalAuth.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["authority"])
            pub_type.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.type"][0]["value"])
           
print(len(pub_type))
print(pub_type)

161
161
['conference paper', 'conference paper', 'journal article', 'conference paper', 'journal article', 'conference paper', 'conference paper', 'conference paper', 'conference paper', 'conference paper', 'conference paper', 'conference paper', 'conference paper', 'conference paper', 'journal article', 'journal article', 'conference paper', 'conference paper', 'patent', 'conference paper', 'patent', 'journal article', 'conference paper', 'conference paper', 'conference paper', 'journal article', 'conference paper', 'journal article', 'conference paper', 'conference paper', 'conference paper', 'journal article', 'conference paper', 'conference paper', 'editorial', 'conference paper', 'journal article', 'conference paper', 'journal article', 'patent', 'journal article', 'conference paper', 'conference paper', 'conference paper', 'journal article', 'journal article', 'conference paper', 'journal article', 'journal article', 'conference paper', 'conference paper', 'conference paper', 'jo

In [ ]:
my_df = pd.DataFrame(pub_type)
my_df.columns = ['Publikationsart']
my_df

,Publikationsart
0,conference paper
1,conference paper
2,journal article
3,conference paper
4,journal article
...,...
156,annual report
157,conference paper
158,conference paper
159,conference paper


In [ ]:
my_df['Publikationsart'].value_counts()

conference paper    90
journal article     62
patent               7
editorial            1
annual report        1
Name: Publikationsart, dtype: int64

In [ ]:
# Neuen Dataframe definieren.
types_counts = my_df['Publikationsart'].value_counts()
df_publicationtypes = pd.DataFrame(types_counts.reset_index())
df_publicationtypes.columns = ['Publikationstyp', 'Anzahl']

df_publicationtypes

,Publikationstyp,Anzahl
0,conference paper,90
1,journal article,62
2,patent,7
3,editorial,1
4,annual report,1


In [ ]:
base = alt.Chart(df_publicationtypes).encode(
    theta=alt.Theta("Anzahl:Q", stack=True), color=alt.Color("Publikationstyp:N")
)

pie = base.mark_arc(outerRadius=120)
text = base.mark_text(radius=140, size=15).encode(text="Anzahl:N")

(pie + text).properties (title='Institutsveröffentlichungen')

alt.LayerChart(...)